In [ ]:
# TM 좌표계 변환 라이브러리
!pip install pyproj

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
pd.options.display.max_columns = None

# data_preprocessing_O

In [ ]:
path = '/content/drive/MyDrive/kt_aivle/big_project/data/used/'

In [ ]:
okay_kids = pd.read_excel(path + '서울키즈_오케이존.xlsx')
okay_kids_categories = pd.read_csv(path + '서울키즈_오케이존_카테고리추출용.csv', encoding = 'CP949')
parents_vip = pd.read_excel(path + '서울엄마아빠VIP존.xlsx')
forests_kids = pd.read_excel(path + '서울_유아숲_체험시설.xlsx')

/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:241: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:241: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:241: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
# 스마트 서울맵 데이터 처리

def preprocessing_cols_names(df):
    df.columns = [col.split('\n')[0] for col in df.columns]
    return df

def cols_update(df, add_drop_cols = []):
    drop_cols = ['공간객체타입', '라인 색상', '라인 패턴', '라인두께', '면 색상', '면 패턴', '면 패턴크기'] # 불필요한 컬럼
    drop_cols += add_drop_cols
    df.drop(columns=drop_cols, inplace=True)    # 불필요한 컬럼
    df.dropna(axis=1, how='all', inplace = True)    # 전체값이 NaN인 경우

    #####  상세 제목의 값이 unique 값이 아닌 것들이 있어서 수정. ####

    title_columns = [col for col in df.columns if '상세 제목' in col]
    content_columns = [col for col in df.columns if '상세 내용' in col]

    new_df = pd.DataFrame()

    for title_col, content_col in zip(title_columns, content_columns):
        # '상세 제목' 컬럼의 값들을 가져옵니다.
        titles = df[title_col]
        contents = df[content_col]

        # 각각의 셀에 대해 새로운 컬럼을 만듭니다.
        for i in range(len(titles)):
            title = titles[i]
            content = contents[i]
            if title not in new_df.columns:
                new_df[title] = [None] * len(df)
            new_df.at[i, title] = content

    # 기존의 '상세 제목'과 '상세 내용' 컬럼을 삭제합니다.
    df = df.drop(columns=title_columns + content_columns)

    # 기존 데이터프레임과 새로운 데이터프레임을 합칩니다.
    df = pd.concat([df, new_df], axis=1)

    return df

### 서울키즈오케이존

In [ ]:
okay_kids = preprocessing_cols_names(okay_kids)
okay_kids = cols_update(okay_kids)

In [ ]:
# Y : 삼성각, 엔제리너스\nL7홍대점, 엔제리너스\n명동점, 대운설렁탕, 샘밭막국수, 바다목장\n바닷가재, 엔제리너스\n롯데백소공식당가점

okay_kids[okay_kids['사용유무'] == 'N']

,콘텐츠 ID,사용유무,콘텐츠명,시군,구명,새주소[도로명 주소],지번주소,키워드,다국어,좌표[X],좌표[Y],전화번호,웹URL,사진 정보1[URL or 파일명],사진 정보2[URL or 파일명],인증사용여부,키즈메뉴(아이가먹을수있는 메뉴),NaN,유아용품,영업시간,상세설명,주차가능여부,주소
38,서초구_016,N,삼성각,서울특별시,서초구,서울특별시 서초구 서초대로78길 42,서울특별시 서초구 서초동 1330-18,"#어린이,#가족",KOR,127.028240,37.494030,02-598-7779,NaN,/smgis2/file/ucimgs/conts/1669595282433/성동구_01...,/smgis2/file/ucimgs/conts/1669595282433/성동구_01...,N,"짜장면,탕수육,볶음밥",None,"유아체어1개,유아식기류",평일(월-금) 11:00~21:30 토일 11:00~21:00,NaN,"0 \n (요리류 주문시 점심 1시간, 저녁2시간 무료)","서울특별시 서초구 서초대로78길 42 (서초동,2층)"
42,강동구_010,N,나주곰탕\n(길동점),서울특별시,강동구,서울특별시 강동구 천호대로 1134,서울특별시 강동구 성내동 379,"#어린이,#가족",KOR,127.137300,37.534010,0507-1337-2966,NaN,/smgis2/file/ucimgs/conts/1669595282433/성동구_01...,/smgis2/file/ucimgs/conts/1669595282433/성동구_01...,N,곰탕,None,유아의자1개,연중무휴 24시간,NaN,0,서울특별시 강동구 천호대로1134
66,서초구_014,N,비푸스,서울특별시,서초구,서울특별시 서초구 서운로26길 11,서울특별시 서초구 서초동 1302-37,"#어린이,#가족",KOR,127.023040,37.502670,02-6092-8575,NaN,/smgis2/file/ucimgs/conts/1669595282433/성동구_01...,/smgis2/file/ucimgs/conts/1669595282433/성동구_01...,N,"뇨끼,3가지버섯&양파버거",None,"유아체어2개,유아식기류",평일(월-금) 카페 8:00~20:00 브런치 08:00~11:00 ...,NaN,"X, 불가","서울특별시 서초구 서운로26길 11, 1층 (서초동)"
69,관악구_017,N,엔제리너스 사당점,서울특별시,관악구,서울특별시 관악구 과천대로 947,서울특별시 관악구 남현동 1061-23,"#어린이,#가족",KOR,126.981369,37.475098,02-523-1937,https://www.krispykreme.co.kr/,/smgis2/file/ucimgs/conts/1669595282433/성동구_01...,/smgis2/file/ucimgs/conts/1669595282433/성동구_01...,N,"미니도넛,\n어린이 음료",None,유아의자,"평일, 주말, 공휴일 \n08:00~22:00",NaN,X,서울특별시 관악구 과천대로 947 사당타워 나동 1층
85,성북구_014,N,셰프박연탄불고기,서울특별시,성북구,서울특별시 성북구 동소문로 305-1,서울특별시 성북구 길음동 510-173,"#어린이,#가족",KOR,127.028350,37.607440,02-983-0433,NaN,/smgis2/file/ucimgs/conts/1669595282433/성동구_01...,/smgis2/file/ucimgs/conts/1669595282433/성동구_01...,N,만두국,None,유아용품 비치예정\n(유아식기류 비치),"평일, 주말, 공휴일\n10:00~21:00\n화요일 휴무",제로페이,X,서울특별시 성북구 동소문로305-1 1층
92,양천구_013,N,엔제리너스\nL7홍대점,서울특별시,마포구,서울특별시 마포구 양화로 141,서울특별시 마포구 동교동 160-5,"#어린이,#가족",KOR,126.921670,37.555750,02-326-0793,https://www.angelinus.com/,/smgis2/file/ucimgs/conts/1669595282433/성동구_01...,/smgis2/file/ucimgs/conts/1669595282433/성동구_01...,N,생과일 주스,None,"유아용의자 1개\n포크, 나이프 구비","평일, 주말, 공휴일\n07:00~24:00",NaN,0,서울특별시 마포구 양화로 141
94,중랑구_029,N,미스터아구신내점,서울특별시,중랑구,서울특별시 중랑구 봉화산로 250,서울특별시 중랑구 신내동 385-7,"#어린이,#가족",KOR,127.100570,37.608160,02-433-5626,NaN,/smgis2/file/ucimgs/conts/1669595282433/성동구_01...,/smgis2/file/ucimgs/conts/1669595282433/성동구_01...,N,돈까스,None,유아의자 3개,"평일, 주말, 공휴일\n 11:30 ~ 22:30",맛있는 음식을 소중한 사람들과 함께 먹는 것처럼 즐거운 일은 없는 것 같습니다.\n...,O,서울특별시 중랑구 봉화산로 250
276,중구_009,N,엔제리너스\n명동점,서울특별시,중구,서울특별시 중구 명동3길 10,서울특별시 중구 명동1가 59-12,"#어린이,#가족",KOR,126.983710,37.564130,02-756-4391,https://www.angelinus.com/,/smgis2/file/ucimgs/conts/1669595282433/성동구_01...,/smgis2/file/ucimgs/conts/1669595282433/성동구_01...,N,생과일 주스,None,"유아용의자 1개\n포크, 나이프 구비","평일 08:00~21:00\n주말, 공휴일 10:00~21:00",NaN,X,서울특별시 중구 명동3길 10
279,중랑구_028,N,능이버섯백숙,서울특별시,중랑구,서울특별시 중랑구 공릉로 32,서울특별시 중랑구 묵동 171-4,"#어린이,#가족",KOR,127.078271,37.613756,02-971-2162,NaN,/smgis2/file/ucimgs/conts/1669595282433/성동구_01...,/smgis2/file/ucimgs/conts/1669595282433/성동구_01...,N,물만두,None,유아의자 2개,"평일, 주말, 공휴일\n 11:00 ~ 22:00",안녕하세요.\n저희 능이버섯백숙은 일반 음식점에서 느낄 수 없는깊은 맛을 자랑하며 ...,O,서울특별시 중랑구 묵동 171-4(묵동공감대아파트)
288,노원구_016,N,아웃백스테이크하우스코리아노원점(노원구 2번과 중복),서울특별시,노원구,서울특별시 노원구 동일로 1419,서울특별시 노원구 상계동 711-12,"#어린이,#가족",KOR,127.059870,37.655320,0507-1308-5471,0,/smgis2/file/ucimgs/conts/1669595282433/성동구_01...,/smgis2/file/ucimgs/conts/1669595282433/성동구_01...,N,블랙라벨 \n패밀리세트 등,None,유아체어10개,"평일(월-금) 11:00~22:00\n토,일 11:00~22:00",NaN,0,서울특별시 노원구 동일로 1419


In [ ]:
# 현재 사용유무 N인 것들 중에서, 현재 사실 확인 후 처리

open_contents = [
    '삼성각', '엔제리너스\nL7홍대점', '엔제리너스\n명동점',
    '대운설렁탕', '샘밭막국수', '바다목장\n바닷가재', '엔제리너스\n롯데백소공식당가점'
]

okay_kids.loc[okay_kids['콘텐츠명'].isin(open_contents) & (okay_kids['사용유무'] == 'N'), '사용유무'] = 'Y'

In [ ]:
okay_kids = okay_kids[okay_kids['사용유무'] != 'N']
okay_kids['콘텐츠명'] = okay_kids['콘텐츠명'].apply(lambda x : x.replace('\n', ' '))
okay_kids.drop(columns = ['사용유무', '다국어', '사진 정보1[URL or 파일명]',
                          '사진 정보2[URL or 파일명]', '인증사용여부', '주소', okay_kids.columns[17]], inplace = True)
okay_kids

<ipython-input-72-764b2afec9ad>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  okay_kids['콘텐츠명'] = okay_kids['콘텐츠명'].apply(lambda x : x.replace('\n', ' '))


,콘텐츠 ID,콘텐츠명,시군,구명,새주소[도로명 주소],지번주소,키워드,좌표[X],좌표[Y],전화번호,웹URL,키즈메뉴(아이가먹을수있는 메뉴),유아용품,영업시간,상세설명,주차가능여부
0,강북구_003,대물림남원추어탕,서울특별시,강북구,서울특별시 강북구 도봉로 192,서울특별시 강북구 미아동 121-27,"#어린이,#가족",127.026700,37.625963,02-945-1292,NaN,삼계탕,유아의자1개,"평일, 주말, 공휴일\n11:00~22:00",가족이 함게 즐기는 직접 만드는 맛깔스런 한끼,0
1,용산구_009,롯데리아 숙대입구역,서울특별시,용산구,서울특별시 용산구 한강대로 283,서울특별시 용산구 갈월동 89-9,"#어린이,#가족",126.971943,37.543368,02-718-9861,https://www.lotteria.com/,"감자튀김, 치즈스틱\n햄버거, 오렌지주스 등","유아용의자2개보유\n포크,보유,나이프\n구비",연중무휴 24시간,부모와 아이들이 모두 즐거운 곳.\n맛있는 즐거움 롯데리아입니다.,X
2,양천구_016,목동카츠,서울특별시,양천구,서울특별시 양천구 목동로9길 4,서울특별시 양천구 신정동 1008-16,"#어린이,#가족",126.864100,37.523050,02-2039-0635,NaN,"등심돈가스, 우동, \n뽀로로어린이음료",유아의자 1개\n포크수저세트,"평일, 주말, 공휴일\n11:30~21:30","제주흑돼지 전문\n돈가스, 우동, 음료",0
3,강동구_009,참나무본가,서울특별시,강동구,서울특별시 강동구 고덕로 421,서울특별시 강동구 강일동 727,"#어린이,#가족",127.172552,37.557418,02-477-6785,NaN,돼지갈비,유아의자4개,"평일, 주말, 공휴일\n11:30~22:30",NaN,0
4,마포구_011,KKD 메세나폴리스점,서울특별시,마포구,서울특별시 마포구 양화로 45,서울특별시 마포구 서교동 490,"#어린이,#가족",126.913769,37.550747,02-3143-5803,https://www.krispykreme.co.kr/,"미니도넛,\n어린이 음료",유아용 의자 2개 비치,"평일, 주말, 공휴일\n08:00~22:00",NaN,O\n(홈플러스 주차장)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
607,노원구_023,미가양꼬치,서울특별시,노원구,서울특별시 노원구 석계로 31,서울특별시 노원구 월계동 73-13,"#어린이,#가족",127.063880,37.617530,NaN,NaN,None,None,None,None,None
608,서초구_017,풍년참숯불갈비,서울특별시,서초구,서울특별시 서초구 서초대로74길 29,서울특별시 서초구 서초동 1327-29,"#어린이,#가족",127.027440,37.494950,02-566-6630,NaN,"돼지갈비,계란찜","돼지갈비,계란찜","평일, 주말, 공휴일\n10:00~22:00\n브레이크타임 15:00~16:00",NaN,0\n (1시간 무료)
609,중랑구_033,대흥정육식당,서울특별시,중랑구,서울특별시 중랑구 중랑천로10길 24,서울특별시 중랑구 면목동 186-149,"#어린이,#가족",127.074480,37.591790,02-435-6288,NaN,계란찜,유아의자,"평일, 주말, 공휴일\n10:00 ~ 22:00",국내산 재료만을 사용하여 온 가족이 안심하고 먹을 수 있는 식당,O
610,강서구_020,엔제리너스 김포공항1층,서울특별시,강서구,서울특별시 강서구 하늘길 177,서울특별시 강서구 공항동 1373-5,"#어린이,#가족",126.808473,37.556756,02-2667-0108,https://www.angelinus.com/,생과일 주스,"유아용의자 1개\n포크, 나이프 구비","평일, 주말, 공휴일\n05:30~22:00",NaN,X


In [ ]:
okay_kids.isna().sum()

콘텐츠 ID                 0
콘텐츠명                   0
시군                     6
구명                     5
새주소[도로명 주소]            0
지번주소                   0
키워드                    6
좌표[X]                  0
좌표[Y]                  0
전화번호                  54
웹URL                 354
키즈메뉴(아이가먹을수있는 메뉴)     33
유아용품                  32
영업시간                  34
상세설명                 333
주차가능여부                56
dtype: int64

In [ ]:
okay_kids_cat = okay_kids_categories[['cot_conts_id', 'cot_sub_cate_name']].copy()
okay_kids_cat.columns = ['콘텐츠 ID', '중분류']
okay_kids_cat['콘텐츠 ID'] = okay_kids_cat['콘텐츠 ID'].apply(lambda x : x.replace('\n', ' '))
okay_kids_cat['대분류'] = '먹기'
okay_kids_cat['실내_실외'] = '실내'
okay_kids_cat

,콘텐츠 ID,중분류,대분류,실내_실외
0,중랑구_041,양식,먹기,실내
1,중랑구_040,한식,먹기,실내
2,중랑구_039,한식,먹기,실내
3,중랑구_038,한식,먹기,실내
4,중랑구_037,한식,먹기,실내
...,...,...,...,...
607,강남구_006,중식,먹기,실내
608,강남구_005,한식,먹기,실내
609,강남구_004,한식,먹기,실내
610,강남구_002,한식,먹기,실내


In [ ]:
preprocessed_okay_kids = pd.merge(okay_kids, okay_kids_cat, how = 'left', on = '콘텐츠 ID')
preprocessed_okay_kids.drop(columns = '콘텐츠 ID', inplace = True)
preprocessed_okay_kids.head(3)

,콘텐츠명,시군,구명,새주소[도로명 주소],지번주소,키워드,좌표[X],좌표[Y],전화번호,웹URL,키즈메뉴(아이가먹을수있는 메뉴),유아용품,영업시간,상세설명,주차가능여부,중분류,대분류,실내_실외
0,대물림남원추어탕,서울특별시,강북구,서울특별시 강북구 도봉로 192,서울특별시 강북구 미아동 121-27,"#어린이,#가족",127.026700,37.625963,02-945-1292,NaN,삼계탕,유아의자1개,"평일, 주말, 공휴일\n11:00~22:00",가족이 함게 즐기는 직접 만드는 맛깔스런 한끼,0,한식,먹기,실내
1,롯데리아 숙대입구역,서울특별시,용산구,서울특별시 용산구 한강대로 283,서울특별시 용산구 갈월동 89-9,"#어린이,#가족",126.971943,37.543368,02-718-9861,https://www.lotteria.com/,"감자튀김, 치즈스틱\n햄버거, 오렌지주스 등","유아용의자2개보유\n포크,보유,나이프\n구비",연중무휴 24시간,부모와 아이들이 모두 즐거운 곳.\n맛있는 즐거움 롯데리아입니다.,X,패스트푸드,먹기,실내
2,목동카츠,서울특별시,양천구,서울특별시 양천구 목동로9길 4,서울특별시 양천구 신정동 1008-16,"#어린이,#가족",126.864100,37.523050,02-2039-0635,NaN,"등심돈가스, 우동, \n뽀로로어린이음료",유아의자 1개\n포크수저세트,"평일, 주말, 공휴일\n11:30~21:30","제주흑돼지 전문\n돈가스, 우동, 음료",0,한식,먹기,실내


In [ ]:
preprocessed_okay_kids['주차가능여부'].value_counts()

주차가능여부
0                                        240
X                                        112
O                                        107
x                                         18
Ο                                          7
○                                          5
×\n주변주차\n가능                                4
ㅇ                                          4
o                                          4
O\n(유료)                                    2
O\n(3대)                                    2
O\n(7대)                                    2
08:00~02:00                                1
가능\n(20대, 발렛)                              1
0,X                                        1
발렛파킹                                       1
�                                          1
O\n(5대)                                    1
   0 \n (1대당 만원이상 주문시 2시간 무료)              1
O\n3시간 무료                                  1
O\n1대 주차가능                                 1
O\n(6대)                                    1
0, 

In [ ]:
preprocessed_okay_kids['주차가능여부'].fillna('X', inplace = True)

def preprocessing_parking(x):
    x = x.split('\n')[0]
    x = x.replace(' ', '')

    values_O = ['0', 'o', 'ο', '○', 'ㅇ', '가능', '백화점', '4대', '지하6층,지하7층', '발렛파킹', '0,유료', '08:00~02:00', '0,X']

    if x in values_O:
        return 'O'
    else:
        return 'X'

preprocessed_okay_kids['주차가능여부'] = preprocessed_okay_kids['주차가능여부'].apply(preprocessing_parking)

In [ ]:
preprocessed_okay_kids['주차가능여부'].value_counts()

주차가능여부
X    333
O    269
Name: count, dtype: int64

In [ ]:
preprocessed_okay_kids.isna().sum()

콘텐츠명                   0
시군                     6
구명                     5
새주소[도로명 주소]            0
지번주소                   0
키워드                    6
좌표[X]                  0
좌표[Y]                  0
전화번호                  54
웹URL                 354
키즈메뉴(아이가먹을수있는 메뉴)     33
유아용품                  32
영업시간                  34
상세설명                 333
주차가능여부                 0
중분류                    0
대분류                    0
실내_실외                  0
dtype: int64

In [ ]:
preprocessed_okay_kids['시군'] = preprocessed_okay_kids['새주소[도로명 주소]'].apply(lambda x : x.split(' ')[0])
preprocessed_okay_kids['구명'] = preprocessed_okay_kids['새주소[도로명 주소]'].apply(lambda x : x.split(' ')[1])

In [ ]:
preprocessed_okay_kids

,콘텐츠명,시군,구명,새주소[도로명 주소],지번주소,키워드,좌표[X],좌표[Y],전화번호,웹URL,키즈메뉴(아이가먹을수있는 메뉴),유아용품,영업시간,상세설명,주차가능여부,중분류,대분류,실내_실외
0,대물림남원추어탕,서울특별시,강북구,서울특별시 강북구 도봉로 192,서울특별시 강북구 미아동 121-27,"#어린이,#가족",127.026700,37.625963,02-945-1292,NaN,삼계탕,유아의자1개,"평일, 주말, 공휴일\n11:00~22:00",가족이 함게 즐기는 직접 만드는 맛깔스런 한끼,O,한식,먹기,실내
1,롯데리아 숙대입구역,서울특별시,용산구,서울특별시 용산구 한강대로 283,서울특별시 용산구 갈월동 89-9,"#어린이,#가족",126.971943,37.543368,02-718-9861,https://www.lotteria.com/,"감자튀김, 치즈스틱\n햄버거, 오렌지주스 등","유아용의자2개보유\n포크,보유,나이프\n구비",연중무휴 24시간,부모와 아이들이 모두 즐거운 곳.\n맛있는 즐거움 롯데리아입니다.,X,패스트푸드,먹기,실내
2,목동카츠,서울특별시,양천구,서울특별시 양천구 목동로9길 4,서울특별시 양천구 신정동 1008-16,"#어린이,#가족",126.864100,37.523050,02-2039-0635,NaN,"등심돈가스, 우동, \n뽀로로어린이음료",유아의자 1개\n포크수저세트,"평일, 주말, 공휴일\n11:30~21:30","제주흑돼지 전문\n돈가스, 우동, 음료",O,한식,먹기,실내
3,참나무본가,서울특별시,강동구,서울특별시 강동구 고덕로 421,서울특별시 강동구 강일동 727,"#어린이,#가족",127.172552,37.557418,02-477-6785,NaN,돼지갈비,유아의자4개,"평일, 주말, 공휴일\n11:30~22:30",NaN,O,한식,먹기,실내
4,KKD 메세나폴리스점,서울특별시,마포구,서울특별시 마포구 양화로 45,서울특별시 마포구 서교동 490,"#어린이,#가족",126.913769,37.550747,02-3143-5803,https://www.krispykreme.co.kr/,"미니도넛,\n어린이 음료",유아용 의자 2개 비치,"평일, 주말, 공휴일\n08:00~22:00",NaN,X,제과,먹기,실내
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
597,미가양꼬치,서울특별시,노원구,서울특별시 노원구 석계로 31,서울특별시 노원구 월계동 73-13,"#어린이,#가족",127.063880,37.617530,NaN,NaN,None,None,None,None,X,중식,먹기,실내
598,풍년참숯불갈비,서울특별시,서초구,서울특별시 서초구 서초대로74길 29,서울특별시 서초구 서초동 1327-29,"#어린이,#가족",127.027440,37.494950,02-566-6630,NaN,"돼지갈비,계란찜","돼지갈비,계란찜","평일, 주말, 공휴일\n10:00~22:00\n브레이크타임 15:00~16:00",NaN,O,한식,먹기,실내
599,대흥정육식당,서울특별시,중랑구,서울특별시 중랑구 중랑천로10길 24,서울특별시 중랑구 면목동 186-149,"#어린이,#가족",127.074480,37.591790,02-435-6288,NaN,계란찜,유아의자,"평일, 주말, 공휴일\n10:00 ~ 22:00",국내산 재료만을 사용하여 온 가족이 안심하고 먹을 수 있는 식당,X,한식,먹기,실내
600,엔제리너스 김포공항1층,서울특별시,강서구,서울특별시 강서구 하늘길 177,서울특별시 강서구 공항동 1373-5,"#어린이,#가족",126.808473,37.556756,02-2667-0108,https://www.angelinus.com/,생과일 주스,"유아용의자 1개\n포크, 나이프 구비","평일, 주말, 공휴일\n05:30~22:00",NaN,X,카페,먹기,실내


In [ ]:
preprocessed_okay_kids.to_excel('/content/drive/MyDrive/kt_aivle/big_project/data/preprocessed_data/preprocessed_okay_kids.xlsx', index=False)

### 서울_유아숲_체험시설

In [ ]:
forests_kids = pd.read_excel(path + '서울_유아숲_체험시설.xlsx')

/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:241: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
forests_kids = preprocessing_cols_names(forests_kids)
forests_kids = cols_update(forests_kids, add_drop_cols = ['사진 정보1[URL or 파일명]', '사진 정보2[URL or 파일명]', '사진 정보3[URL or 파일명]', '사진 정보4[URL or 파일명]', '사진 정보5[URL or 파일명]'])
forests_kids = forests_kids[forests_kids['콘텐츠명'].str.contains('숲체험')]
forests_kids = forests_kids[forests_kids['사용유무'] != 'N'][['콘텐츠명', '구명', '새주소[도로명 주소]', '지번주소', '키워드', '좌표[X]', '좌표[Y]', '전화번호', '주요시설']]
forests_kids['중분류'] = '숲체험'
forests_kids['대분류'] = '놀기'
forests_kids['실내_실외'] = '실외'
forests_kids
# 숲체험, 및 유아숲 관련 키워드 정리
forests_kids['키워드'] = '학습능력, 창의성, 집중력, 탐구능력, 환경감수성, 자연, 심리안정, 숲, 자아존중감, 숲체험, 숲, 체험학습'

In [ ]:
forests_kids.to_excel('/content/drive/MyDrive/kt_aivle/big_project/data/preprocessed_data/preprocessed_forests_kids.xlsx', index=False)

In [ ]:
# 유아숲체험원 사이트 : https://parks.seoul.go.kr/forestkid/index.htm
# 2024년 유아숲체험원 현황 : https://opengov.seoul.go.kr/public/30139599
# 현재 처리한 스마트맵 유아숲_테마 몇개 내용이 업데이트가 되지않아서, 이후 추가로 엑셀로 작업

### 서울형 키즈카페 시설현황정보

* 서울형 키즈카페 정보 : https://icare.seoul.go.kr/icare/dolbomMENU5/dolbomMENU5_1.jsp



In [ ]:
seoul_kidscafe = pd.read_csv(path + '서울형 키즈카페 시설현황정보.csv', encoding = 'euc-kr')
seoul_kidscafe = seoul_kidscafe[['시설명', '서비스분류명', '자치구명', '행정동명', 'x좌표값', 'y좌표값', '기본주소', '연락처', '사용료무료여부','운영일', '휴관일', '이용정원(기관)', '이용정원(개인)', '신청가능연령']]

In [ ]:
# 6.20일 기준 서울형 키즈카페 31개. 1개 추가하기.
seoul_kidscafe[seoul_kidscafe['시설명'].str.contains('서울')]['시설명']

0            서울형 키즈카페 중랑구 망우본동점(중랑실내놀이터 양원)
1      서울형 키즈카페 송파구 잠실근린공원점(하하호호놀이터 송파구1호점)
2                        서울형 키즈카페 시립 뚝섬자벌레점
3                      서울형 키즈카페 중구 중림점 노리몽땅
4                     서울형 키즈카페 성북구 벌집어린이공원점
5                         서울형 키즈카페 관악구 난곡동점
6               서울형 키즈카페 강동구 성내1동점(아이?맘 강동)
7               서울형 키즈카페 도봉구 창1동점(숲속유람선 뚜뚜)
8                서울형 키즈카페 도봉구 방학1동점(오르봉내리봉)
9               서울형 키즈카페 강동구 상일2동점(아이?맘 강동)
10                       서울형 키즈카페 양천구 오목공원점
11        서울형 키즈카페 서초구 양재1동점(서리풀노리학교 양재1동점)
12                          서울형 키즈카페 시립 목동점
13                서울형 키즈카페 도봉구 도봉2동점(봉봉트레킹)
14    서울형 키즈카페 서초구 문화예술공원점(서리풀노리학교 문화예술공원점)
15        서울형 키즈카페 용산구 한강로동점(용산 도담도담 실내놀이터)
16                      서울형 키즈카페 영등포구 대림2동점
17                    서울특별시 제1호 시립 서울형 키즈카페
18                       서울형 키즈카페 강서구 발산1동점
19                        서울형 키즈카페 마포구 상암동점
20                서울형 키즈카페 동작구 대방동점(동작키즈카페)
21         서울형 키즈카페 강북구 번3동점(강북구 'PLAY ON')
22             서울형 키즈카페 광진구 중곡3동

In [ ]:
# 이용료 무료 수정
# 무료 : idx : 2 - 시설명 : 서울형 키즈카페 시립 뚝섬자벌레점'

# 시설명 수정 : ? -> ·
seoul_kidscafe.loc[(seoul_kidscafe['시설명'] != '서울형 키즈카페 시립 뚝섬자벌레점'), '사용료무료여부'] = 'N'
seoul_kidscafe['시설명'] = seoul_kidscafe['시설명'].str.replace('?', '·')

In [ ]:
new_seoul_kidscafe = {
    '시설명': '서울형 키즈카페 서초구 방배2동점(서리풀노리학교 방배2동점)',
    '서비스분류명': '서울형키즈카페',
    '자치구명': '서초구',
    '행정동명': '방배동',
    'x좌표값': 126.983798,
    'y좌표값': 37.484555,
    '기본주소': '서울특별시 서초구 방배천로24길 8 방주교회',
    '연락처': '02-523-9340',
    '사용료무료여부': 'N',
    '운영일': '월-토',
    '휴관일': '일요일, 근로자의 날, 법정공휴일',
    '이용정원(기관)': '18명(영유아기준)',
    '이용정원(개인)': '15명(영유아기준)',
    '신청가능연령': '0세 ~ 6세'
}
new_seoul_kidscafe_df = pd.DataFrame([new_seoul_kidscafe])

seoul_kidscafe = pd.concat([seoul_kidscafe, new_seoul_kidscafe_df], ignore_index=True)

In [ ]:
update_list = ['서울형키즈카페중랑실내놀이터 양원점', '하하호호 놀이터 송파구1호점', '서울형 키즈카페 시립 뚝섬자벌레점', '노리몽땅', '서울형 키즈카페 성북구 벌집어린이공원점',
                '서울형 키즈카페 관악구 난곡동점', '서울형키즈카페 성내1호점', '숲속유람선뚜뚜', '오르봉내리봉', '서울형키즈카페 상일2동점',
                '서울형키즈카페 오목공원점', '서리풀노리학교 문화예술공원점', '시립양천거점형키움센터,서울형키즈카페', '서울형키즈카페 도봉구도봉2동점', '서리풀노리학교 문화예술공원점',
                '서울형키즈카페 용산구한강로동점', '서울형키즈카페 영등포구대림2동점', '서울특별시 제1호 시립 서울형 키즈카페', '서울형키즈카페 강서구 발산1동점',
                '서울형키즈카페 마포구상암점', '서울형키즈카페 동작구대방동점', '서울형 키즈카페 강북구 번3동점 강북구 PLAY ON', '서울형 키즈카페 광진구 중곡 3동점', '서울형키즈카페 암사1동점',
                '서울형키즈카페 강동구 고덕2동점', '서울형키즈카페 성동구금호2.3가동점', '서울형키즈카페 상도3동점', '서울형키즈카페 양천구 신정7동점 5색깔깔KIDS', '서울형 키즈카페 중랑구 면목4동점',
                '서울형키즈카페 종로구혜화동점', '서울형 키즈카페 서초구 방배2동점']
seoul_kidscafe.loc[:, '시설명'] = update_list


In [ ]:
seoul_kidscafe['중분류'] = '공공형/서울형 키즈카페'
seoul_kidscafe['대분류'] = '놀기'
seoul_kidscafe['실내_실외'] = '실내'

In [ ]:
seoul_kidscafe.to_excel('/content/drive/MyDrive/kt_aivle/big_project/data/preprocessed_data/preprocessed_seoul_kidscafe.xlsx', index=False)

### 서울시 유원시설업(기타) 인허가 정보

In [ ]:
seoul_policy = pd.read_csv(path + '서울시 유원시설업(기타) 인허가 정보.csv', encoding = 'euc-kr')
seoul_policy = seoul_policy.loc[seoul_policy['영업상태명'] == '영업/정상']

In [ ]:
# https://map.naver.com/p/entry/place/11781740?lng=127.1213374&lat=37.5485167&placePath=%2Fhome&entry=plt&searchType=place&c=15.00,0,0,0,dh
# 광나루 수영장 : 2025년 까지 재조성 공사로 인하여 영업 X
# 삭제
display(seoul_policy[seoul_policy['상세영업상태명'].isna()])

seoul_policy.drop(index = [566, 567], inplace = True)

,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태코드,영업상태명,상세영업상태코드,상세영업상태명,폐업일자,휴업시작일자,휴업종료일자,재개업일자,전화번호,소재지면적,소재지우편번호,지번주소,도로명주소,도로명우편번호,사업장명,최종수정일자,데이터갱신구분,데이터갱신일자,업태구분명,좌표정보(X),좌표정보(Y),문화체육업종명,문화사업자구분명,지역구분명,총층수,주변환경명,제작취급품목내용,보험기관명,건물용도명,지상층수,지하층수,객실수,건축연면적,영문상호명,영문상호주소,선박총톤수,선박척수,선박제원,무대면적,좌석수,기념품종류,회의실별동시수용인원,시설면적,놀이기구수내역,놀이시설수,방송시설유무,발전시설유무,의무실유무,안내소유무,기획여행보험시작일자,기획여행보험종료일자,자본금,보험시작일자,보험종료일자,부대시설내역,시설규모
566,3240000,CDFI5272002012000002,20120716,NaN,1,영업/정상,BBBB,NaN,NaN,NaN,NaN,NaN,470-9561,NaN,134876,서울특별시 강동구 암사동 637-6번지,서울특별시 강동구 선사로 83-66 (암사동),134876.0,광나루 수영장,2014-03-25 17:26:10,I,2018-08-31 23:59:59.0,NaN,210734.925941,449757.057853,신고유원시설업,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99.0
567,3240000,CDFI5272002014000003,20140711,NaN,1,영업/정상,BBBB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,134876,서울특별시 강동구 암사동 637-6번지 광나루수영장,"서울특별시 강동구 선사로 83-66 (암사동, 광나루수영장)",134876.0,민테크,2014-09-24 14:04:40,I,2018-08-31 23:59:59.0,NaN,210734.925941,449757.057853,신고유원시설업,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# 서울형키즈카페 idx 제거
# 목록 : 1, 47, 93, 99,
display(seoul_policy[seoul_policy['사업장명'].str.contains('서울')]['사업장명'])

seoul_policy.drop(index = [1, 47, 93, 99], inplace = True)

1                 서울형 키즈카페 강서구 발산1동점
47     마포구육아종합지원센터 내 서울형키즈카페 마포구 상암점
93                    서울형 키즈카페 대림2동점
99                   서울형 키즈카페(중곡3동점)
109                         5층, 에픽서울
235     서울특별시 강남구 도시관리공단(세곡어린이실내놀이터)
446                      닥터밸런스 서울서부점
447                          리틀 서울랜드
485                Hello Park 서울 남현점
Name: 사업장명, dtype: object

In [ ]:
seoul_policy = seoul_policy.loc[: ,['전화번호', '지번주소', '도로명주소', '사업장명', '좌표정보(X)', '좌표정보(Y)']]

In [ ]:
"""
사업장명 처리 목록
---------------------------------------------------

idx 7, 사업장명 : 꼬마노리터 -> 꼬마대통령 천호점
idx 11, 사업장명 : 뛰노리방방 -> 뛰노리방방 방학점
idx 22, 사업장명 : 더블유에스컴퍼니 -> 데굴데굴키즈파티룸 용답점
idx 27, 사업장명 : (주) 헬로방방 서초본점 -> 헬로방방 서초점
idx 29, 사업장명 : (주)플레이스케이프 -> 플레이스케이프 롯데몰김포공항점
idx 33, 사업장명 : 스트라이크팡 신촌2-> 스트라이크팡 신촌점
idx 49, 사업장명 : 라온아띠 유한회사 (월드킹키즈카페)구로점 -> 월드킹 구로점
idx 53, 사업장명 : 주식회사 카르디아(팜시티) -> 팜시티
idx 62, 사업장명 : 주식회사 원더브라더스3(스마트브릭 키즈카페)-> 스마트브릭 키즈카페
idx 78, 사업장명 : 제이블리 -> 제이블리키즈카페 목동점
idx 85, 사업장명 : (주)세라젬웰파크 롯데월드점 -> 세라젬 웰파크 잠실점
idx 87, 사업장명 : 주식회사 블루디벨롭 -> 골든블루키즈빌리지
idx 95, 사업장명 : 문화철도959(토카아트) -> 문화철도959
idx 102, 사업장명 : (주)이키즈랜드 코코몽 송파점 -> 코코몽키즈랜드 송파점
idx 109, 사업장명 : 5층, 에픽서울 -> 현대백화점 더현대 서울
idx 110, 사업장명 : 펀키즈(Funkids) -> Funkids
idx 111, 사업장명 : 짱오락실 강남대로점(주식회사짱) -> 짱오락실 강남대로점
idx 115, 사업장명 : 주식회사 짱(짱이네오락실 강남2호점) -> 짱오락실 강남2호점
idx 117, 사업장명 : 리틀비틀 키즈카페 -> 리틀비틀 장안점
idx 120, 사업장명 : 에듀퐁(쁘띠몽드) -> 쁘띠몽드 장안점
idx 130, 사업장명 : 꼬마빌리지 -> 꼬마빌리지 월곡점
idx 136, 사업장명 : 캘리클럽X태그액션 더익스트림 아이파크몰 고척점 -> 캘리클럽 고척점
idx 138, 사업장명 : (주)뽀로로파크 빅마켓 영등포점 -> 뽀로로파크 롯데마트맥스 영등포점
idx 143, 사업장명 : 짱오락실 -> 짱오락실 홍대점
idx 148, 사업장명 : (주)바운스 트램폴린파크 -> 바운스트램폴린파크 반포센터
idx 149, 사업장명 : 펀시티 -> 펀시티 건대점
idx 159, 사업장명 : 주식회사 짱 -> 짱오락실 상암동 메가박스점
idx 204, 사업장명 : 플레이타임중앙(주) 용산아이파크몰 챔피언1250 -> 챔피언1250X 아이파크몰 용산점
idx 235, 사업장명 : 서울특별시 강남구 도시관리공단(세곡어린이실내놀이터) -> 세곡어린이실내놀이터
idx 256, 사업장명 : 뛰노리방방 -> 뛰노리방방 번동본점
idx 262, 사업장명 : 키즈카페 점프점프 -> 원어민영어키즈카페 점프점프
idx 290, 사업장명 : 헬로방방 돌곶이역점 (대표남편:) -> 헬로방방 돌곶이역점
idx 309, 사업장명 : 링커스영배 -> 볼베어파크 롯데몰 은평점
idx 320	, 사업장명 : ㈜ 바운스 트램폴린파크(월계 이마트센터) -> 바운스 트램폴린파크 월계이마트센터
idx 346, 사업장명 : 타이거릴리 -> 타이거릴리 현대시티몰 가든파이브점
idx 374, 사업장명 : 리틀마운틴 영등포점 -> 리틀마운틴 롯데백화점 영등포점
idx 387, 사업장명 : 헬로방방 -> 헬로방방 목동점
idx 400, 사업장명 : 꼬꼬마놀이터 -> 꼬꼬마 실내놀이터
idx 414, 사업장명 : 롤리폴리 트렘폴린 -> 롤리폴리 강서화곡점
idx 415, 사업장명 : 위드키즈 -> 키즈카페 위드키즈
idx 416, 사업장명 : 챔피언 더 블랙벨트 -> 챔피언더블랙벨트 롯데몰 김포공항점
idx 425, 사업장명 : (주)굿플레이 -> 굿플레이 롯데백화점 김포공항점
idx 444, 사업장명 : 퐁퐁플라워 서쪽나라 -> 퐁퐁플라워 롯데아울렛 가산센터
idx 445, 사업장명 : 금천구육아종합지원센터 독산점 -> 금천구육아종합지원센터 독산점 아이세상놀이터
idx 452, 사업장명 : 뛰노리방방 -> 뛰노리방방 고척점
idx 475, 사업장명 : 퐁퐁플라워 팁스오리진 -> 퐁퐁플라워 롯데백화점 관악센터
idx 490, 사업장명 : 주식회사 원더브라더스 -> 원더볼즈플레이파크
idx 507, 사업장명 : (주)바운스(바운스 트램폴린 삼성센터-VAUNCE) -> 바운스 트램폴린파크 삼성센터
idx 508, 사업장명 : 아이레 -> 아이레키즈카페
idx 510, 사업장명 : 주식회사 텐씨엘(CALI CLUB) -> 캘리클럽 청담점
idx 540, 사업장명 : (주)키드존 -> 키즈카페


(주) 가 있다면, (주) 제거
홈플러스 가 있다면, 홈플러스


사업장명 삭제 목록
---------------------------------------------------
idx 17, 서울과기대 - 여름행사 - 사업장명 : 노원 SUMMER 워터파크 -> 23년 1달간 행사
idx 37, 아파트
idx 44
idx 55
idx 68
idx 83
idx 86 - 서울형 키즈카페
idx 92. 서울과기대 - 겨울 행사 - 썰매장
idx 122	- 서울형 키즈카페
idx 123
idx 131
idx 134
idx 152
idx 153
idx 166
idx 166
idx 187
idx 219
idx 263
idx 265
idx 273
idx 280
idx 286
idx 321
idx 323
idx 324
idx 328
idx 341
idx 342
idx 343
idx 385
idx 386
idx 410
idx 411
idx 412
idx 413
idx 419
idx 420
idx 422
idx 424
idx 467

사업장 카테고리 목록
---------------------------------------------------
사업장명 : 보물찾기(홍대점), 스트라이크팡 신촌점, 팜시티, 포토 앤 게임, 스포츠게임, 짱오락실 강남대로점, 짱오락실 강남2호점,
                   짱오락실 홍대점, 펀시티 건대점, 짱오락실 상암동 메가박스점, 맥스라이더 신내점 -> 오락실


# 특색있는 키워드 카페 목록 사업장명 .contains('코코몽', '타요', '뽀로로')

# 강남구청에서 지원하는 공공형실내놀이터 : https://www.gangnam.go.kr/contents/memewe_clean_intro/1/view.do?mid=ID04_02071901#
# idx : 150, 235, 525
"""

seoul_policy[['지번주소', '도로명주소', '사업장명']]

,지번주소,도로명주소,사업장명
2,서울특별시 구로구 구로동 573 NC백화점,"서울특별시 구로구 구로중앙로 152, NC백화점 7층 (구로동)",(주)이키즈랜드
3,서울특별시 강동구 천호동 42 천호복합 상업시설,"서울특별시 강동구 양재대로 1571, 천호복합 상업시설 2층 (천호동)",블럭앤퍼즐애플
4,서울특별시 서초구 서초동 1684-1 서초쇼핑,"서울특별시 서초구 고무래로 90-7, 서초쇼핑 지하1층 101호, 102호 (서초동)",점핑파크 서초점
5,서울특별시 강동구 명일동 306-7,"서울특별시 강동구 양재대로 1657-11, 3층 (명일동, 대은상가)",프레리키즈카페(명일동점)
7,서울특별시 강동구 천호동 453-18,"서울특별시 강동구 천호대로157길 6, 2층 (천호동)",꼬마노리터
...,...,...,...
554,서울특별시 송파구 방이동 227-4 오투존 스포츠센터,"서울특별시 송파구 오금로 235, 오투존 스포츠센터 5층 (방이동)",헬로방방 송파방이점
555,서울특별시 송파구 방이동 141-6 3층,"서울특별시 송파구 백제고분로48길 31, 3층층 (방이동)",위드키즈 키즈카페
556,서울특별시 송파구 잠실동 40-1 롯데마트 5층,"서울특별시 송파구 올림픽로 240, 롯데마트 5층 (잠실동)",슈퍼윙스 스포츠파크 롯데 잠실점
564,서울특별시 강동구 천호동 454-1 이마트,"서울특별시 강동구 천호대로 1017, 이마트 지하2층 (천호동)",리틀마운틴 천호점


In [ ]:
# 사업장명 처리 목록
update_list = {
    7: '꼬마대통령 천호점',
    11: '뛰노리방방 방학점',
    22: '데굴데굴키즈파티룸 용답점',
    27: '헬로방방 서초점',
    29: '플레이스케이프 롯데몰김포공항점',
    33: '스트라이크팡 신촌점',
    49: '월드킹 구로점',
    53: '팜시티',
    62: '스마트브릭 키즈카페',
    78: '제이블리키즈카페 목동점',
    85: '세라젬 웰파크 잠실점',
    87: '골든블루키즈빌리지',
    95: '문화철도959',
    102: '코코몽키즈랜드 송파점',
    109: '현대백화점 더현대 서울',
    110: 'Funkids',
    111: '짱오락실 강남대로점',
    115: '짱오락실 강남2호점',
    117: '리틀비틀 장안점',
    120: '쁘띠몽드 장안점',
    130: '꼬마빌리지 월곡점',
    136: '캘리클럽 고척점',
    138: '뽀로로파크 롯데마트맥스 영등포점',
    143: '짱오락실 홍대점',
    148: '바운스트램폴린파크 반포센터',
    149: '펀시티 건대점',
    159: '짱오락실 상암동 메가박스점',
    204: '챔피언1250X 아이파크몰 용산점',
    235: '세곡어린이실내놀이터',
    256: '뛰노리방방 번동본점',
    262: '원어민영어키즈카페 점프점프',
    290: '헬로방방 돌곶이역점',
    309: '볼베어파크 롯데몰 은평점',
    320: '바운스 트램폴린파크 월계이마트센터',
    346: '타이거릴리 현대시티몰 가든파이브점',
    374: '리틀마운틴 롯데백화점 영등포점',
    387: '헬로방방 목동점',
    400: '꼬꼬마 실내놀이터',
    414: '롤리폴리 강서화곡점',
    415: '키즈카페 위드키즈',
    416: '챔피언더블랙벨트 롯데몰 김포공항점',
    425: '굿플레이 롯데백화점 김포공항점',
    444: '퐁퐁플라워 롯데아울렛 가산센터',
    445: '금천구육아종합지원센터 독산점 아이세상놀이터',
    452: '뛰노리방방 고척점',
    475: '퐁퐁플라워 롯데백화점 관악센터',
    490: '원더볼즈플레이파크',
    507: '바운스 트램폴린파크 삼성센터',
    508: '아이레키즈카페',
    510: '캘리클럽 청담점',
    540: '키즈카페'
}

# 사업장명 처리
for idx, new_name in update_list.items():
    if idx in seoul_policy.index:
        seoul_policy.loc[idx, '사업장명'] = new_name

# 사업장명 (주) 가 있다면 (주) 제거,
seoul_policy['사업장명'] = seoul_policy['사업장명'].str.replace(r'\(주\)', '', regex=True).str.strip()
# 사업장명 홈플러스 가 포함되어있으면, 홈플러스
seoul_policy['사업장명'] = seoul_policy['사업장명'].apply(lambda x: '홈플러스' if '홈플러스' in x else x)



# 데이터 행 삭제 목록
delete_list = [17, 37, 44, 55, 68, 83, 86, 92, 122, 123, 131, 134, 152, 153, 166, 187, 219, 263, 265, 273, 280, 286, 321, 323, 324, 328, 341, 342, 343, 385, 386, 410, 411, 412, 413, 419, 420, 422, 424, 467]

seoul_policy.drop(index = delete_list, inplace = True)

In [ ]:
from pyproj import Proj, Transformer

# 한국 중부 원점(Korea Central Origin) TM 좌표계 정의 (EPSG:5174)
proj_tm_mid = Proj(init='epsg:5174')

# WGS84 좌표계 정의 (EPSG:4326)
proj_wgs84 = Proj(init='epsg:4326')

transformer = Transformer.from_crs("epsg:5174", "epsg:4326")

def TM_to_WGS84(x, y):
    # lon, lat = transform(proj_tm_mid, proj_wgs84, x, y)
    lon, lat = transformer.transform(x, y)
    return lat, lon

seoul_policy[['x좌표값', 'y좌표값']] = seoul_policy.apply(lambda x: pd.Series(TM_to_WGS84(x['좌표정보(X)'], x['좌표정보(Y)'])), axis=1)
seoul_policy.drop(columns = ['좌표정보(X)', '좌표정보(Y)'], inplace = True)

/usr/local/lib/python3.10/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.10/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [ ]:
seoul_policy.loc[150, 'x좌표값'] = 127.074170
seoul_policy.loc[150, 'y좌표값'] = 37.491094

seoul_policy.loc[159, 'x좌표값'] = 126.897273
seoul_policy.loc[159, 'y좌표값'] = 37.568288

In [ ]:
"""
추가 삭제

idx 28, 사업장명 : 올림픽팡팡 -> 야외 / 키즈카페
idx 32, 사업장명 : 동작주차공원 물놀이장 -> 수영장
idx 113, 사업장명 : 로라스로라월드(주) -> 키즈카페 / 롤러스케이트장
idx 319, 사업장명 : 날개미술창작소 -> 공방
28, 32, 113, 319
"""

seoul_policy.drop(index = [28, 32, 113, 319], inplace = True)

In [ ]:
seoul_policy['중분류'] = '민간 키즈카페'
seoul_policy['대분류'] = '놀기'
seoul_policy['실내_실외'] = '실내'

update_cat_list = ['보물찾기(홍대점)', '스트라이크팡 신촌점', '팜시티', '포토 앤 게임', '스포츠게임', '짱오락실 강남대로점', '짱오락실 강남2호점', '짱오락실 홍대점', '펀시티 건대점', '짱오락실 상암동 메가박스점', '맥스라이더 신내점']

seoul_policy.loc[seoul_policy['사업장명'].isin(update_cat_list), '중분류'] = '오락실'

# 강남구청에서 운영하는 공공형 실내 놀이터 idx : 150, 235, 525
seoul_policy.loc[[150, 235, 525], '중분류'] = '공공형/서울형 키즈카페'

In [ ]:
seoul_policy.to_excel('/content/drive/MyDrive/kt_aivle/big_project/data/preprocessed_data/preprocessed_seoul_policy.xlsx', index=False)